In [65]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [ ]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [57]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7028, 8)

In [58]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6734, 5)

In [59]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7034, 10)

fin empresas y comienzo de rescate de pacientes


In [60]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1637649, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [61]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1210043, 4)

In [62]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1210043, 13)

In [63]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [64]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [ ]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [ ]:
#pacientes_empresas22.shape

In [ ]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [ ]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [41]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BICE Vida"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [42]:
Pacientes_Archivo.head(10)

,Empresa *,rut1,sexo,nombres,apellidos,Fecha Nacimiento*,asDSAFD,SFDSGFG,GFDHFGHJFGJH,nacimiento,division
0,BRG SPORTS - MTY,CORL910905MSPVVZ10,F,LUZ MARIA,COVARRUBIAS RIVERA,1991-09-05 00:00:00,5,9,1991,1991-09-05,Mty
1,BRG SPORTS - MTY,BOLL890730MCLRNR08,F,LAURA ELIZABETH,BROWN DE LUNA,1989-07-30 00:00:00,30,7,1989,1989-07-30,Mty
2,BRG SPORTS - MTY,LAPJ900812HNLRRR00,M,JORGE OBED,LARA PEREZ,12/Ago/90,12,8,1990,1990-08-12,Mty
3,BRG SPORTS - MTY,REVF791021HCLNRR08,M,FERNANDO DAVID,RENTERIA VARGAS,1979-10-21 00:00:00,21,10,1979,1979-10-21,Mty
4,BRG SPORTS - MTY,BABF760426HJCXGR08,M,FERMIN,BAÑUELOS BUGARIN,26/Abr/76,26,4,1976,1976-04-26,Mty
5,BRG SPORTS - MTY,ZURC780716MZSXZR04,F,MARIA DEL CARMEN,ZUÑIGA RUIZ,1978-07-16 00:00:00,16,7,1978,1978-07-16,Mty
6,BRG SPORTS - MTY,GAMB800217MNLRRL00,F,BLANCA ESTHELA,GARCIA MARTINEZ,1980-02-17 00:00:00,17,2,1980,1980-02-17,Mty
7,BRG SPORTS - MTY,TOIL780109HNLRBS04,M,JOSE LUIS,TORRES IBARRA,09/Ene/78,9,1,1978,1978-01-09,Mty
8,BRG SPORTS - MTY,RORJ670607MTSCDN03,F,JUANA MARIA,ROCHA RODRIGUEZ,1967-06-07 00:00:00,7,6,1967,1967-06-07,Mty
9,BRG SPORTS - MTY,YEMI791121HNLPDV02,M,JOSE IVAN,YEPEZ MEDINA,1979-11-21 00:00:00,21,11,1979,1979-11-21,Mty


In [43]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [44]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Empresa *,rut1,sexo,nombres,apellidos,Fecha Nacimiento*,asDSAFD,SFDSGFG,GFDHFGHJFGJH,nacimiento,division,rut
0,BRG SPORTS - MTY,CORL910905MSPVVZ10,F,LUZ MARIA,COVARRUBIAS RIVERA,1991-09-05 00:00:00,5,9,1991,1991-09-05,Mty,CORL910905MSPVVZ10
1,BRG SPORTS - MTY,BOLL890730MCLRNR08,F,LAURA ELIZABETH,BROWN DE LUNA,1989-07-30 00:00:00,30,7,1989,1989-07-30,Mty,BOLL890730MCLRNR08
2,BRG SPORTS - MTY,LAPJ900812HNLRRR00,M,JORGE OBED,LARA PEREZ,12/Ago/90,12,8,1990,1990-08-12,Mty,LAPJ900812HNLRRR00
3,BRG SPORTS - MTY,REVF791021HCLNRR08,M,FERNANDO DAVID,RENTERIA VARGAS,1979-10-21 00:00:00,21,10,1979,1979-10-21,Mty,REVF791021HCLNRR08
4,BRG SPORTS - MTY,BABF760426HJCXGR08,M,FERMIN,BAÑUELOS BUGARIN,26/Abr/76,26,4,1976,1976-04-26,Mty,BABF760426HJCXGR08


In [45]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1676\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa *,rut1,sexo,nombres,apellidos,Fecha Nacimiento*,asDSAFD,SFDSGFG,GFDHFGHJFGJH,nacimiento,division,rut
0,BRG SPORTS - MTY,CORL910905MSPVVZ10,F,LUZ MARIA,COVARRUBIAS RIVERA,1991-09-05 00:00:00,5,9,1991,1991-09-05,Mty,CORL910905MSPVVZ10
1,BRG SPORTS - MTY,BOLL890730MCLRNR08,F,LAURA ELIZABETH,BROWN DE LUNA,1989-07-30 00:00:00,30,7,1989,1989-07-30,Mty,BOLL890730MCLRNR08
2,BRG SPORTS - MTY,LAPJ900812HNLRRR00,M,JORGE OBED,LARA PEREZ,12/Ago/90,12,8,1990,1990-08-12,Mty,LAPJ900812HNLRRR00
3,BRG SPORTS - MTY,REVF791021HCLNRR08,M,FERNANDO DAVID,RENTERIA VARGAS,1979-10-21 00:00:00,21,10,1979,1979-10-21,Mty,REVF791021HCLNRR08
4,BRG SPORTS - MTY,BABF760426HJCXGR08,M,FERMIN,BAÑUELOS BUGARIN,26/Abr/76,26,4,1976,1976-04-26,Mty,BABF760426HJCXGR08


In [46]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1676\1486029825.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1676\1486029825.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [47]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(379, 22)

In [48]:
Pacientes_con_id.head()

,Empresa *,rut1,sexo,nombres,apellidos,Fecha Nacimiento*,asDSAFD,SFDSGFG,GFDHFGHJFGJH,nacimiento,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,BRG SPORTS - MTY,CORL910905MSPVVZ10,F,Luz Maria,Covarrubias Rivera,1991-09-05 00:00:00,5,9,1991,1991-09-05,...,True,0018c00002ivuR0AAI,CORL910905MSPVVZ10,Luz Maria Covarrubias Rivera,CURP,None,None,Mty,Mujer,None
1,BRG SPORTS - MTY,BOLL890730MCLRNR08,F,Laura Elizabeth,Brown De Luna,1989-07-30 00:00:00,30,7,1989,1989-07-30,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BRG SPORTS - MTY,LAPJ900812HNLRRR00,M,Jorge Obed,Lara Perez,12/Ago/90,12,8,1990,1990-08-12,...,True,0018c00002ivuR1AAI,LAPJ900812HNLRRR00,Jorge Obed Lara Perez,CURP,None,None,Mty,Hombre,None
3,BRG SPORTS - MTY,REVF791021HCLNRR08,M,Fernando David,Renteria Vargas,1979-10-21 00:00:00,21,10,1979,1979-10-21,...,True,0018c00002ivuR2AAI,REVF791021HCLNRR08,Fernando David Renteria Vargas,CURP,None,None,Mty,Hombre,None
4,BRG SPORTS - MTY,BABF760426HJCXGR08,M,Fermin,Bañuelos Bugarin,26/Abr/76,26,4,1976,1976-04-26,...,True,0018c00002ivuR3AAI,BABF760426HJCXGR08,Fermin Bañuelos Bugarin,CURP,None,None,Mty,Hombre,None


In [49]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,HealthCloudGA__Gender__pc,PersonDepartment,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
1,False,Laura Elizabeth,Brown De Luna,BOLL890730MCLRNR08,1989-07-30,Mujer,Mty,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
51,False,Brandon Jair,Armijo Velazquez,AIVB961207HNLRLR05,1996-12-07,Hombre,Mty,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
74,False,Emilio,Lopez Cruz,LOCE970218HCSPRM02,1997-02-18,Hombre,Mty,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
88,False,Ediberto,Mendoza Hernandez,MEHE990319HCSNRD07,1999-03-19,Hombre,Mty,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE
119,False,Noe Javier,Rodriguez Garcia,ROGN921110HVZDRX00,1992-11-10,Hombre,Mty,0128c000002Byp4AAC,México,CURP,a1U8c000005dAtsEAE


In [50]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [51]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
591522,a1Y8c00000DoX2dEAF,0018c00002ivlBrAAI,a1W8c0000083sauEAA,Riddell Sports,0018c00002gl023AAA,Riddell Sports MX,None,Riddell Sports,Riddell Sports MX,None,None,EMP10267,A-9827
598005,a1Y8c00000DoZKTEA3,0018c00002gkpqzAAA,a1W8c0000083sauEAA,Riddell Sports,0018c00002gl023AAA,Riddell Sports MX,None,Riddell Sports,Riddell Sports MX,None,None,EMP10267,A-9827
598007,a1Y8c00000DoZKYEA3,0018c00002glMHEAA2,a1W8c0000083sauEAA,Riddell Sports,0018c00002gl023AAA,Riddell Sports MX,None,Riddell Sports,Riddell Sports MX,None,None,EMP10267,A-9827
598823,a1Y8c00000Doa2GEAR,0018c00002glMEwAAM,a1W8c0000083sauEAA,Riddell Sports,0018c00002gl023AAA,Riddell Sports MX,None,Riddell Sports,Riddell Sports MX,None,None,EMP10267,A-9827
598832,a1Y8c00000Doa3NEAR,0018c00002kCdQeAAK,a1W8c0000083sauEAA,Riddell Sports,0018c00002gl023AAA,Riddell Sports MX,None,Riddell Sports,Riddell Sports MX,None,None,EMP10267,A-9827


In [52]:
campaña_seleccionada.shape

(748, 13)

In [53]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [55]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [ ]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
